In [20]:
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
df = spark.read.format("csv").option("inferschema","True").option("header","True").option("sep",",").load("netflix_list.csv")

In [2]:
display(df)

DataFrame[imdb_id: string, title: string, certificate: string, startYear: int, type: string, origin_country: string, language: string, rating: double, numVotes: double, genres: string]

In [3]:
df.show(5)

+----------+--------------------+-----------+---------+-----+--------------+--------+------+--------+--------------------+
|   imdb_id|               title|certificate|startYear| type|origin_country|language|rating|numVotes|              genres|
+----------+--------------------+-----------+---------+-----+--------------+--------+------+--------+--------------------+
| tt0993840|    Army of the Dead|         18|     2021|movie| United States| English|   5.8|110780.0| Action,Crime,Horror|
| tt6111574|The Woman in the ...|         18|     2021|movie| United States| English|   5.7| 46703.0| Crime,Drama,Mystery|
| tt7979580|The Mitchells vs ...|         13|     2021|movie| United States| English|   7.8| 55762.0|Adventure,Animati...|
| tt7084386|        Blue Miracle|         13|     2021|movie| United States| English|   6.7|  2962.0|Adventure,Biograp...|
|tt10962368| Things Heard & Seen|         16|     2021|movie| United States| English|   5.3| 20762.0|Drama,Horror,Mystery|
+----------+----

In [4]:
df.columns


['imdb_id',
 'title',
 'certificate',
 'startYear',
 'type',
 'origin_country',
 'language',
 'rating',
 'numVotes',
 'genres']

In [5]:
df.select('title','genres').where(df['genres'].isin(['Crime,Drama,Mystery'])).show()

+--------------------+-------------------+
|               title|             genres|
+--------------------+-------------------+
|The Woman in the ...|Crime,Drama,Mystery|
|      I Am All Girls|Crime,Drama,Mystery|
+--------------------+-------------------+



In [6]:
df.select('certificate').distinct().show()

+-----------+
|certificate|
+-----------+
|          7|
|         16|
|       null|
|         PG|
|         18|
|          U|
|        All|
|          A|
|          R|
|          G|
|         13|
|      PG-13|
+-----------+



In [7]:
df.select('type').distinct().show()

+-----+
| type|
+-----+
|movie|
+-----+



In [40]:
df.select('language').distinct().show()

+--------+
|language|
+--------+
| English|
+--------+



In [9]:
df.crosstab('title','certificate').select('title_certificate','7','16','null','PG','18','U','All','A','R','G','13','PG-13').show()

+--------------------+---+---+----+---+---+---+---+---+---+---+---+-----+
|   title_certificate|  7| 16|null| PG| 18|  U|All|  A|  R|  G| 13|PG-13|
+--------------------+---+---+----+---+---+---+---+---+---+---+---+-----+
|Rise of the Teena...|  0|  0|   1|  0|  0|  0|  0|  0|  0|  0|  0|    0|
|       Farewell Amor|  0|  0|   1|  0|  0|  0|  0|  0|  0|  0|  0|    0|
|The App That Stol...|  0|  0|   1|  0|  0|  0|  0|  0|  0|  0|  0|    0|
|  Single All the Way|  0|  0|   1|  0|  0|  0|  0|  0|  0|  0|  0|    0|
|Happiness Ever After|  0|  0|   1|  0|  0|  0|  0|  0|  0|  0|  0|    0|
|Secrets of the Sa...|  0|  0|   0|  0|  0|  0|  0|  0|  0|  0|  1|    0|
|Bob Ross: Happy A...|  0|  0|   1|  0|  0|  0|  0|  0|  0|  0|  0|    0|
|Mucho Mucho Amor:...|  0|  0|   0|  0|  0|  0|  0|  0|  0|  0|  1|    0|
|The Last Days of ...|  0|  0|   0|  0|  1|  0|  0|  0|  0|  0|  0|    0|
|Have a Good Trip:...|  0|  0|   1|  0|  0|  0|  0|  0|  0|  0|  0|    0|
|United Nation Thr...|  0|  0|   1|  0

In [10]:
df.select('genres').show()

+--------------------+
|              genres|
+--------------------+
| Action,Crime,Horror|
| Crime,Drama,Mystery|
|Adventure,Animati...|
|Adventure,Biograp...|
|Drama,Horror,Mystery|
|Drama,Sci-Fi,Thri...|
|              Comedy|
| Crime,Drama,Mystery|
|Adventure,Animati...|
|Biography,Drama,H...|
|Action,Adventure,...|
|            Thriller|
|Action,Adventure,...|
|      Horror,Mystery|
|Crime,Documentary...|
|Action,Adventure,...|
|  Comedy,Drama,Music|
|       Comedy,Family|
|Action,Adventure,...|
|Action,Adventure,...|
+--------------------+
only showing top 20 rows



In [11]:
df.crosstab('type','genres').select('type_genres','Action,Crime,Horror','Crime,Drama,Mystery').show()

+-----------+-------------------+-------------------+
|type_genres|Action,Crime,Horror|Crime,Drama,Mystery|
+-----------+-------------------+-------------------+
|      movie|                  2|                  2|
+-----------+-------------------+-------------------+



In [12]:
from pyspark.sql.functions import *


In [13]:
df = df.na.fill({'rating':0.0}).show()


+----------+--------------------+-----------+---------+-----+--------------+--------+------+--------+--------------------+
|   imdb_id|               title|certificate|startYear| type|origin_country|language|rating|numVotes|              genres|
+----------+--------------------+-----------+---------+-----+--------------+--------+------+--------+--------------------+
| tt0993840|    Army of the Dead|         18|     2021|movie| United States| English|   5.8|110780.0| Action,Crime,Horror|
| tt6111574|The Woman in the ...|         18|     2021|movie| United States| English|   5.7| 46703.0| Crime,Drama,Mystery|
| tt7979580|The Mitchells vs ...|         13|     2021|movie| United States| English|   7.8| 55762.0|Adventure,Animati...|
| tt7084386|        Blue Miracle|         13|     2021|movie| United States| English|   6.7|  2962.0|Adventure,Biograp...|
|tt10962368| Things Heard & Seen|         16|     2021|movie| United States| English|   5.3| 20762.0|Drama,Horror,Mystery|
| tt9203694|    

In [26]:
from pyspark.sql.types import DataType
print(df.schema["genres"].dataType)

StringType


In [23]:
from pyspark.sql.functions import col
df.filter(col("genres").contains("Crime")).show()


+----------+--------------------+-----------+---------+-----+--------------+--------+------+--------+--------------------+
|   imdb_id|               title|certificate|startYear| type|origin_country|language|rating|numVotes|              genres|
+----------+--------------------+-----------+---------+-----+--------------+--------+------+--------+--------------------+
| tt0993840|    Army of the Dead|         18|     2021|movie| United States| English|   5.8|110780.0| Action,Crime,Horror|
| tt6111574|The Woman in the ...|         18|     2021|movie| United States| English|   5.7| 46703.0| Crime,Drama,Mystery|
| tt9013182|      I Am All Girls|         16|     2021|movie|  South Africa| English|   5.8|  3283.0| Crime,Drama,Mystery|
|tt12939812|      The Nailbomber|       null|     2021|movie|United Kingdom| English|   6.3|  1107.0|Crime,Documentary...|
| tt7991608|          Red Notice|       null|     2021|movie| United States| English|  null|    null| Action,Comedy,Crime|
|tt13024674|    

In [24]:
crime_movie = df.filter(col("genres").contains("Crime"))

crime_movie.select('title','rating').show()

+--------------------+------+
|               title|rating|
+--------------------+------+
|    Army of the Dead|   5.8|
|The Woman in the ...|   5.7|
|      I Am All Girls|   5.8|
|      The Nailbomber|   6.3|
|          Red Notice|  null|
|     Army of Thieves|  null|
|          The Guilty|  null|
|                Kate|  null|
|Operation Varsity...|   7.0|
|Crack: Cocaine, C...|   6.7|
|        I Care a Lot|   6.3|
|The Devil All the...|   7.1|
|        Honest Thief|   6.0|
|        Enola Holmes|   6.6|
|Spenser Confidential|   6.2|
|       Project Power|   6.0|
|     Force of Nature|   4.5|
|The Last Thing He...|   4.3|
|The Last Days of ...|   3.7|
|American Murder: ...|   7.2|
+--------------------+------+
only showing top 20 rows



In [38]:
crime_movie.count()

24

In [27]:
split_col = split(df['genres'], ',')

In [28]:
df2 = df.withColumn('genre_type_1', split_col.getItem(0)) \
       .withColumn('genre_type_2', split_col.getItem(1)) \
       .withColumn('genre_type_3', split_col.getItem(2))
df2.show(truncate=False)   

+----------+---------------------------------+-----------+---------+-----+--------------+--------+------+--------+--------------------------+------------+------------+------------+
|imdb_id   |title                            |certificate|startYear|type |origin_country|language|rating|numVotes|genres                    |genre_type_1|genre_type_2|genre_type_3|
+----------+---------------------------------+-----------+---------+-----+--------------+--------+------+--------+--------------------------+------------+------------+------------+
|tt0993840 |Army of the Dead                 |18         |2021     |movie|United States |English |5.8   |110780.0|Action,Crime,Horror       |Action      |Crime       |Horror      |
|tt6111574 |The Woman in the Window          |18         |2021     |movie|United States |English |5.7   |46703.0 |Crime,Drama,Mystery       |Crime       |Drama       |Mystery     |
|tt7979580 |The Mitchells vs the Machines    |13         |2021     |movie|United States |Englis

In [29]:
df2.select('genre_type_1','genre_type_2','genre_type_3').show()

+------------+------------+------------+
|genre_type_1|genre_type_2|genre_type_3|
+------------+------------+------------+
|      Action|       Crime|      Horror|
|       Crime|       Drama|     Mystery|
|   Adventure|   Animation|      Comedy|
|   Adventure|   Biography|       Drama|
|       Drama|      Horror|     Mystery|
|       Drama|      Sci-Fi|    Thriller|
|      Comedy|        null|        null|
|       Crime|       Drama|     Mystery|
|   Adventure|   Animation|      Comedy|
|   Biography|       Drama|     History|
|      Action|   Adventure|      Comedy|
|    Thriller|        null|        null|
|      Action|   Adventure|    Thriller|
|      Horror|     Mystery|        null|
|       Crime| Documentary|     History|
|      Action|   Adventure|       Drama|
|      Comedy|       Drama|       Music|
|      Comedy|      Family|        null|
|      Action|   Adventure|   Animation|
|      Action|   Adventure|     Fantasy|
+------------+------------+------------+
only showing top

In [30]:
import pandas as pd


In [31]:
pd_df = pd.read_csv("netflix_list.csv")
pd_df.certificate =pd_df.certificate.astype(str)

In [32]:
pd_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 235 entries, 0 to 234
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   imdb_id         235 non-null    object 
 1   title           235 non-null    object 
 2   certificate     235 non-null    object 
 3   startYear       235 non-null    int64  
 4   type            235 non-null    object 
 5   origin_country  235 non-null    object 
 6   language        235 non-null    object 
 7   rating          176 non-null    float64
 8   numVotes        176 non-null    float64
 9   genres          235 non-null    object 
dtypes: float64(2), int64(1), object(7)
memory usage: 18.5+ KB


In [33]:
pd_df =df.na.fill({'numVotes':0.0})

In [34]:
pd_df.show()

+----------+--------------------+-----------+---------+-----+--------------+--------+------+--------+--------------------+
|   imdb_id|               title|certificate|startYear| type|origin_country|language|rating|numVotes|              genres|
+----------+--------------------+-----------+---------+-----+--------------+--------+------+--------+--------------------+
| tt0993840|    Army of the Dead|         18|     2021|movie| United States| English|   5.8|110780.0| Action,Crime,Horror|
| tt6111574|The Woman in the ...|         18|     2021|movie| United States| English|   5.7| 46703.0| Crime,Drama,Mystery|
| tt7979580|The Mitchells vs ...|         13|     2021|movie| United States| English|   7.8| 55762.0|Adventure,Animati...|
| tt7084386|        Blue Miracle|         13|     2021|movie| United States| English|   6.7|  2962.0|Adventure,Biograp...|
|tt10962368| Things Heard & Seen|         16|     2021|movie| United States| English|   5.3| 20762.0|Drama,Horror,Mystery|
| tt9203694|    

In [35]:
pd_df = pd_df.na.fill({'rating':0.0}).show()


+----------+--------------------+-----------+---------+-----+--------------+--------+------+--------+--------------------+
|   imdb_id|               title|certificate|startYear| type|origin_country|language|rating|numVotes|              genres|
+----------+--------------------+-----------+---------+-----+--------------+--------+------+--------+--------------------+
| tt0993840|    Army of the Dead|         18|     2021|movie| United States| English|   5.8|110780.0| Action,Crime,Horror|
| tt6111574|The Woman in the ...|         18|     2021|movie| United States| English|   5.7| 46703.0| Crime,Drama,Mystery|
| tt7979580|The Mitchells vs ...|         13|     2021|movie| United States| English|   7.8| 55762.0|Adventure,Animati...|
| tt7084386|        Blue Miracle|         13|     2021|movie| United States| English|   6.7|  2962.0|Adventure,Biograp...|
|tt10962368| Things Heard & Seen|         16|     2021|movie| United States| English|   5.3| 20762.0|Drama,Horror,Mystery|
| tt9203694|    